<a href="https://colab.research.google.com/github/Chandan-Kumar0211/a2a--Reports/blob/main/pp_yolo_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Mounted at /content/gdrive


In [2]:
import glob
import os

os.chdir('/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/obj')
myFiles = glob.glob('*.txt')
print(len(myFiles))

1104


In [ ]:
import shutil, os
files = myFiles
for f in files:
    shutil.copy(f, '/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/txt_annotation')

In [3]:
import glob
import os

os.chdir('/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/obj')
myFiles = glob.glob('*.jpg')
print(len(myFiles))

1105


In [ ]:
import shutil, os
files = myFiles
for f in files:
    shutil.copy(f, '/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/train_images')

In [ ]:
import os
import cv2
 
# 
originLabelsDir = '/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/txt_annotation'
 # Converted file save path
saveDir = '/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/annos.txt'                                                                       
 # Picture path corresponding to the original label
originImagesDir = '/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/train_images'
 
txtFileList = os.listdir(originLabelsDir)
with open(saveDir, 'w') as fw:
    for txtFile in txtFileList:
        with open(os.path.join(originLabelsDir, txtFile), 'r') as fr:
            labelList = fr.readlines()
            for label in labelList:
                label = label.strip().split()
                x = float(label[1])
                y = float(label[2])
                w = float(label[3])
                h = float(label[4])
 
                # convert x,y,w,h to x1,y1,x2,y2
                imagePath = os.path.join(originImagesDir,
                                         txtFile.replace('txt', 'jpg'))
                image = cv2.imread(imagePath)
                H, W, _ = image.shape
                x1 = (x - w / 2) * W
                y1 = (y - h / 2) * H
                x2 = (x + w / 2) * W
                y2 = (y + h / 2) * H
                                 # In order to match the coco label method, the label serial number is calculated from 1
                fw.write(txtFile.replace('txt', 'jpg') + ' {} {} {} {} {}\n'.format(int(label[0]) + 1, x1, y1, x2, y2))
 
        print('{} done'.format(txtFile))


COCO_train2014_000000188637.txt done
COCO_train2014_000000188454.txt done
COCO_train2014_000000180831.txt done
COCO_train2014_000000180324.txt done
COCO_train2014_000000172751.txt done
COCO_train2014_000000172123.txt done
COCO_train2014_000000172096.txt done
COCO_train2014_000000164287.txt done
COCO_train2014_000000164114.txt done
COCO_train2014_000000163908.txt done
COCO_train2014_000000278673.txt done
COCO_train2014_000000148118.txt done
COCO_train2014_000000278793.txt done
COCO_train2014_000000213537.txt done
COCO_train2014_000000221785.txt done
COCO_train2014_000000221805.txt done
COCO_train2014_000000229522.txt done
COCO_train2014_000000229852.txt done
COCO_train2014_000000229969.txt done
COCO_train2014_000000237922.txt done
COCO_train2014_000000238004.txt done
COCO_train2014_000000238196.txt done
COCO_train2014_000000238290.txt done
COCO_train2014_000000246311.txt done
COCO_train2014_000000246335.txt done
COCO_train2014_000000253975.txt done
COCO_train2014_000000254218.txt done
C

In [ ]:
! cp -r /content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/annos.txt /content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/train_images

In [ ]:
import json
import os
import cv2


# ------------ Use os to extract the image name in the images folder, and read the BBox into it ------------
 #root path, which contains images (picture folder), annos.txt (bbox annotation), classes.txt (category label),
 # and annotations folder (created automatically if not, used to save the last json)

root_path = '/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO'
 # Used to create a training set or validation set
Phase = 'train' # need to be corrected
 
 # dataset is used to save image information and annotation information of all data
dataset = {'categories': [], 'annotations': [], 'images': []}
 
 # Open category label
with open(os.path.join(root_path, 'annos.txt')) as f:
    classes = f.read().strip().split()
 
 # Establish the correspondence between category labels and numeric ids
for i, cls in enumerate(classes, 1):
    dataset['categories'].append({'id': i, 'name': cls, 'supercategory': 'mark'})
 
 # Read the image name of the images folder
indexes = os.listdir(os.path.join(root_path, 'train_images'))
 
 # Statistics Processing the number of pictures
global count
count = 0
 
 # Read Bbox information
with open(os.path.join(root_path, 'annos.txt')) as tr:
    annos = tr.readlines()
 
         # --------------- Then, the above data is converted to the format required by COCO ---------------
    for k, index in enumerate(indexes):
        count += 1
                 # Read images with opencv to get the width and height of the image
        try:
          im = cv2.imread(os.path.join(root_path, 'train_images/') + index)
          height, width, _ = im.shape
  
                  # Add image information to the dataset
          dataset['images'].append({'file_name': index,
                                    'id': k,
                                    'width': width,
                                    'height': height})
  
          for ii, anno in enumerate(annos):
              parts = anno.strip().split()
  
                          # Add a tag if the name of the image and the name of the tag are on
              if parts[0] == index:
                                  # 
                  cls_id = parts[1]
                  # x_min
                  x1 = float(parts[2])
                  # y_min
                  y1 = float(parts[3])
                  # x_max
                  x2 = float(parts[4])
                  # y_max
                  y2 = float(parts[5])
                  width = max(0, x2 - x1)
                  height = max(0, y2 - y1)
                  dataset['annotations'].append({
                      'area': width * height,
                      'bbox': [x1, y1, width, height],
                      'category_id': int(cls_id),
                      'id': i,
                      'image_id': k,
                      'iscrowd': 0,
                                          # mask, the rectangle is the four vertices clockwise from the top left corner
                      'segmentation': [[x1, y1, x2, y1, x2, y2, x1, y2]]
                  })
  
          print('{} images handled'.format(count))
        except:
          print('file ' + os.path.join('/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO', 'obj/') + index + ' is missing, continuing')
          pass
 
 #Save the resulting folder
folder = os.path.join(root_path, 'annotations')
if not os.path.exists(folder):
  os.makedirs(folder)
json_name = os.path.join(root_path, 'annotations/img.json'.format(Phase))
with open(json_name, 'w') as f:
  json.dump(dataset, f)

1 images handled
2 images handled
3 images handled
4 images handled
5 images handled
6 images handled
7 images handled
8 images handled
9 images handled
10 images handled
11 images handled
12 images handled
13 images handled
14 images handled
15 images handled
16 images handled
17 images handled
18 images handled
19 images handled
20 images handled
21 images handled
22 images handled
23 images handled
24 images handled
25 images handled
26 images handled
27 images handled
28 images handled
29 images handled
30 images handled
31 images handled
32 images handled
33 images handled
34 images handled
35 images handled
36 images handled
37 images handled
38 images handled
39 images handled
40 images handled
41 images handled
42 images handled
43 images handled
44 images handled
45 images handled
46 images handled
47 images handled
48 images handled
49 images handled
50 images handled
51 images handled
52 images handled
53 images handled
54 images handled
55 images handled
56 images handled
5

In [4]:
%cd /content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO

/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO


In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

Cloning into 'PaddleDetection'...
remote: Enumerating objects: 14361, done.
remote: Counting objects: 100% (1307/1307), done.
remote: Compressing objects: 100% (682/682), done.
remote: Total 14361 (delta 779), reused 1010 (delta 624), pack-reused 13054
Receiving objects: 100% (14361/14361), 133.65 MiB | 20.71 MiB/s, done.
Resolving deltas: 100% (10116/10116), done.
Checking out files: 100% (1219/1219), done.


In [5]:
pip install paddledet==2.1.0 -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 419kB 190kB/s 
     |████████████████████████████████| 1.5MB 2.1MB/s 
     |████████████████████████████████| 2.7MB 9.5MB/s 
     |████████████████████████████████| 153kB 17.2MB/s 
     |████████████████████████████████| 2.0MB 12.5MB/s 
     |████████████████████████████████| 194kB 18.1MB/s 
     |████████████████████████████████| 204kB 19.5MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 7.2MB 18.6MB/s 
     |████████████████████████████████| 102kB 10.0MB/s 
     |████████████████████████████████| 1.9MB 23.1MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 348kB 27.9MB/s 
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-lin

In [6]:
!python -m pip install paddlepaddle-gpu==2.1.0.post101 -f https://paddlepaddle.org.cn/whl/mkl/stable.html

Looking in links: https://paddlepaddle.org.cn/whl/mkl/stable.html
     |████████████████████████████████| 344.9MB 50kB/s 


In [7]:
!python -c "import paddle; print(paddle.__version__)"

2.1.0


In [8]:
%cd /content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/PaddleDetection

/content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/PaddleDetection


In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!python tools/train.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml

/usr/local/lib/python3.7/dist-packages/IPython/utils/module_paths.py:29: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(key, collections.Hashable):
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
W0703 07:06:11.867098   625 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 11.2, Runtime API Version: 10.1
W0703 07:06:11.873287   625 device_context.cc:422] device: 0, cuDNN Version: 7.6.
[07/03 07:06:16] ppdet.utils.checkpoint INFO: Finish loading model weights: /root/.cache/paddle/weights/ResNet50_vd_ssld_pretrained.pdparams
[07/03 07:06:17] ppdet.engine INFO: Epoch: [0] [  0

In [9]:
#python -u tools/infer.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml  --infer_img=content/gdrive/MyDrive/Colab_Notebooks/PP_YOLO/test_images/images.jfif

SyntaxError: ignored

In [11]:
!python tools/infer.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml -o use_gpu=false weights=output/ppyolo_r50vd_dcn_1x_coco/model_final.pdparams --infer_img=demo/000000570688.jpg --save_txt=true

/usr/local/lib/python3.7/dist-packages/IPython/utils/module_paths.py:29: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(key, collections.Hashable):
[07/03 16:46:26] ppdet.utils.checkpoint INFO: Finish loading model weights: output/ppyolo_r50vd_dcn_1x_coco/model_final.pdparams
[07/03 16:46:29] ppdet.engine INFO: Detection bbox results save in output/000000570688.jpg


In [12]:
!python tools/infer.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml -o use_gpu=false weights=output/ppyolo_r50vd_dcn_1x_coco/model_final.pdparams --infer_img=demo/images.jfif --save_txt=true

/usr/local/lib/python3.7/dist-packages/IPython/utils/module_paths.py:29: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(key, collections.Hashable):
[07/03 16:47:57] ppdet.utils.checkpoint INFO: Finish loading model weights: output/ppyolo_r50vd_dcn_1x_coco/model_final.pdparams
Traceback (most recent call last):
  File "tools/infer.py", line 158, in <module>
    main()
  File "tools/infer.py", line 154, in main
    run(FLAGS, cfg)
  File "tools/infer.py", line 132, in run
    save_txt=FLAGS.save_txt)
  File "/usr/local/lib/python3.7/dist-packages/ppdet/engine/trainer.py", line 395, in predict
    self.dataset.set_images(images)
  File "/usr/local/lib/pyth